In [2]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm 
import re
from gensim.utils import simple_preprocess 
import gensim
from gensim.models.word2vec import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download("vader_lexicon")
from nltk.sentiment import SentimentIntensityAnalyzer

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /opt/conda/nltk_data...


In [2]:
# Read in the dataframe of labeled tweets

tweet_df = pd.read_json('tweet.json')

In [3]:
tweet_df.head()

,ID,tweet,label
0,17461978,[RT @CarnivalCruise: 🎉 Are you ready to see wh...,0
1,1297437077403885568,None,1
2,17685258,[RT @realDonaldTrump: THANK YOU #RNC2020! http...,0
3,15750898,[A family fears they may have been cheated out...,0
4,1659167666,[RT @VonteThePlug: Yeah but he ain’t got one h...,1


In [4]:
tweet_df.shape

(11826, 3)

In [5]:
profile_df = pd.read_json('profile.json')

In [6]:
profile_df.shape

(11826, 3)

Features Extraction

In [7]:
tw_features_df = tweet_df.merge(profile_df, on=['ID', 'label']).dropna()

51

In [8]:
# Number of listed and statuses for each user
tw_features_df['listed_count'] = tw_features_df['profile'].apply(lambda x: int(x['listed_count']))
tw_features_df['statuses_count'] = tw_features_df['profile'].apply(lambda x: int(x['statuses_count']))

# Description length
tw_features_df['description_len'] =  tw_features_df['profile'].apply(lambda x: len(x['description']))

In [40]:
# Average sentiment of user from their tweets
sia = SentimentIntensityAnalyzer()
tw_features_df['sentiment'] = tw_features_df['tweet'].apply(lambda x: np.mean([ sia.polarity_scores(t)['compound'] for t in x ]))
tw_features_df['sentiment'] = tw_features_df['sentiment'].apply(lambda x: x>0) 
tw_features_df['sentiment'] = tw_features_df['sentiment'].replace({True:'Positive',False:'Negative'})

In [47]:
tw_features_df.head()

,ID,tweet,label,profile,listed_count,statuses_count,description_len,sentiment
0,17461978,[RT @CarnivalCruise: 🎉 Are you ready to see wh...,0,"{'id': '17461978 ', 'id_str': '17461978 ', 'na...",45568,9798,51,Positive
2,17685258,[RT @realDonaldTrump: THANK YOU #RNC2020! http...,0,"{'id': '17685258 ', 'id_str': '17685258 ', 'na...",3201,5518,161,Positive
3,15750898,[A family fears they may have been cheated out...,0,"{'id': '15750898 ', 'id_str': '15750898 ', 'na...",1744,192876,161,Positive
4,1659167666,[RT @VonteThePlug: Yeah but he ain’t got one h...,1,"{'id': '1659167666 ', 'id_str': '1659167666 ',...",44,103,104,Negative
5,34743251,"[RT @elonmusk: Aloha, welcome back from space ...",0,"{'id': '34743251 ', 'id_str': '34743251 ', 'na...",26952,4879,91,Positive
